# Wo hat es in Deutschland in dem Jahr, in dem Du geboren bist, gebrannt?

## Gib hier das Jahr an, an dem Du geboren bist. 
Prinzipiell kannst ein Jahr zwischen 2000 und 2020 angeben.

In [1]:
jahr = 2009             

## Nun laden wir die benötigten Apps.

In [2]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append('/sw/spack-rhel6/jupyterhub/jupyterhub/lib/python3.8/site-packages')

In [3]:
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
init_notebook_mode(connected=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Nun holen wir die Datei mit den MODIS Feuerbeobachtungen vom NASA Datenarchiv und lesen sie ein.

In [4]:
land       = 'Germany'
datei      = 'https://firms.modaps.eosdis.nasa.gov/data/country/modis/'+str(jahr)+'/modis_'+str(jahr)+'_Germany.csv'
feuerdaten = pd.read_csv(datei, usecols=['latitude','longitude','acq_date','acq_time','frp','type'])

## Hier passen wir die Daten etwas an und wählen nur Vegetationsbrände aus.
Bei jedem Feuer gibt es eine Typbeschreibung: Handelt es sich wahrscheinlich um ein Vegetationsbrand oder etwas anderes? Wir wählen nur Feuer aus, die vermutlich echte Vegetationsbrände sind.

In [5]:
feuerdaten['Zeitpunkt'] = pd.to_datetime(feuerdaten['acq_date'] + ' ' + feuerdaten['acq_time'].astype(str).str.zfill(4).str.wrap(2).str.replace('\n', ':')) + pd.DateOffset(hours=2)
feuerdaten.drop(['acq_time','acq_date'], axis='columns', inplace=True)
feuerdaten['Datum'] = pd.to_datetime(feuerdaten['Zeitpunkt']).dt.date.astype(str)


feuerdaten.rename({'latitude':'Breitengrad', 'longitude':'Längengrad', 'frp':'Feuerintensität', 'type':'Typ'}, axis='columns', inplace=True)
feuerdaten['Typ'].replace(to_replace=[0, 2], value=['Vegetationsbrand', 'Anderes'],inplace=True)
feuerdaten=feuerdaten[feuerdaten['Typ'] != 'Anderes']

### Wie sehen die Daten nun aus (erste 5 Zeilen)?

In [6]:
feuerdaten.head(5)

,Breitengrad,Längengrad,Feuerintensität,Typ,Zeitpunkt,Datum
1,51.4892,6.7248,11.0,Vegetationsbrand,2009-01-12 03:42:00,2009-01-12
20,49.2923,8.6136,10.2,Vegetationsbrand,2009-02-03 14:16:00,2009-02-03
21,49.2947,8.6279,17.2,Vegetationsbrand,2009-02-03 14:16:00,2009-02-03
27,49.3559,6.7447,6.6,Vegetationsbrand,2009-03-03 12:52:00,2009-03-03
32,51.4659,6.2459,12.7,Vegetationsbrand,2009-03-17 13:04:00,2009-03-17


###  Wie viele Vegetationsbrände gab es in dem Jahr in Deutschland?

In [7]:
print(feuerdaten['Typ'].value_counts().tolist()[0])

671


# Nun zeichnen wir eine Deutschlandkarte mit den Feuern. 

### Hier bestimmen wir die Farben des Farbbalkens.

In [8]:
cmap = [[0, 'black'], [0.5, 'red'],[1, 'yellow']]

Hier könntet ihr auch andere Farben auf Englisch wählen. 
Wichtig ist, dass der englische Farbname klein geschrieben ist.
Ihr könntet auch eine Regenbogenfarbskala verwenden. Dann müsst ihr nur schreiben:<br>
cmap= 'rainbow'

### Hier wird die Abbildung gemacht:

In [9]:
vmin, vmax = 0, 50
fig = px.scatter_mapbox(feuerdaten, 
                        lat = 'Breitengrad', 
                        lon = 'Längengrad', 
                        color = 'Feuerintensität', 
                        size = 'Feuerintensität',
                        color_continuous_scale  = cmap,
                        opacity = 0.5,
                        range_color = [vmin, vmax],
                        hover_data=['Typ'],
                        animation_frame= 'Datum',  
                        center = {'lat': 51.17, 'lon': 10.45}, 
                        zoom = 4.5,
                        mapbox_style = 'carto-positron')
fig.update_layout(
    title='Feuer in Deutschland im Jahr '+str(jahr), title_x=0.5,
    width=700, height=800,
    legend={'yanchor': 'top', 'xanchor':'left', 'y':1, 'x': 0.1},
    coloraxis={'colorbar':{'title':'Feuerintensität [MW]', 'ticks':'inside'}})

fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 20


In [10]:
pio.write_html(fig, file = 'MODIS_Feuer_Deutschland_'+str(jahr)+'.html')